In [3]:
import mxnet as mx

MXNet中一个非常重要的对象是多维数组，也就是mxnet.ndarray或mxnet.nd。

``mxnet.ndarray``和`numpy.ndarray`非常相似。

### The basic

[1,2,3]是一个１维数组，下面是一个１维数组，它第一维度长度是２，第二维度长度是３:

[[0, 1, 2]
 
 [3, 4, 5]]

数组类是NDArray。他有几个重要的属性：
* ndarray.shape, 返回一个tuple，包含各个维度的长度，比如一个n*m的矩阵，shape是(n, m)
* ndarray.dtype, 返回numpy对象描述数组元素的类型
* ndarray.size, 返回数组中元素的个数，值等于shape各个元素的乘积
* ndarray.context, 返回存储这个数组的设备，比如CPU或第ｉ个GPU

#### 创建数组

创建数组的方法有多个。最常用的是array方法，**array方法接受的第一个参数包括Python list、tuple、numpy.ndarray**:

In [4]:
# 从Python list创建一个１维数组
a = mx.nd.array([1,2,3])

# 从Python list 创建一个２维数组
b = mx.nd.array([[1,2,3], [2,3,4]])

print {'a.shape': a.shape, 'b.shape': b.shape}

{'b.shape': (2L, 3L), 'a.shape': (3L,)}


In [5]:
import numpy as np
import math

c = np.arange(15).reshape(3,5)

# 从numpy.ndarray创建一个２维数组
a = mx.nd.array(c)

{'a.shape': a.shape}

{'a.shape': (3L, 5L)}

array方法接收的第二个参数是dtype, 也就是numpy type,　默认，float32:

In [6]:
# 默认使用float32
a = mx.nd.array([1,2,3])
# 创建一个int32数组
b = mx.nd.array([1,2,3], dtype=np.int32)
# 创建一个16bit float数组
c = mx.nd.array([1.2, 2.3], dtype=np.float16)

(a.dtype, b.dtype, c.dtype)

(numpy.float32, numpy.int32, numpy.float16)

**除了使用array方法，有时候我们想创建一个只知道大小(size)，而不知道元素值的数组，mxnet支持创建占位符内容的数组**:

In [7]:
# 创建一个２维数组，元素值都是０，shape=(2,3)
a = mx.nd.zeros((2, 3))

# 创建一个shape同上的数组，不过元素初始值为１
b = mx.nd.ones((2, 3))

# 创建一个shape同上的数组，不过元素初始值为7
c = mx.nd.full((2, 3), 7)

#  创建一个shape同上的数组，不过元素初始值是随机数
d = mx.nd.empty((2, 3))

#### 打印数组值

数组创建好了，如何查看数组值呢？怎么样把值打印出来看看？


使用asnumpy方法把NDArray转换成numpy.ndarray，然后自动打印出数组值:

In [8]:
b = mx.nd.ones((2, 3))
b.asnumpy()

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.]], dtype=float32)

#### 基本操作

两个数组的算数操作方式是elementwise。并且自动创建一个存储计算结果的数组。

In [9]:
a = mx.nd.ones((2, 3))
b = mx.nd.ones((2, 3))

# elementwise plus
c = a + b

# elementwise minus
d = - c

# elementwise pow and sin, and then 转置
e = mx.nd.sin(c**2).T

# elementwise max
f = mx.nd.maximum(a, c)

f.asnumpy()

array([[ 2.,  2.,  2.],
       [ 2.,  2.,  2.]], dtype=float32)

同NumPy, * 作用于elementwise，想要实现矩阵乘法要用dot:

In [10]:
a = mx.nd.ones((2, 2))
b = a * a
c = mx.nd.dot(a, a)

{'b': b.asnumpy(), 'c': c.asnumpy()}

{'b': array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 'c': array([[ 2.,  2.],
        [ 2.,  2.]], dtype=float32)}

**+=, *=等赋值操作是in place operation，不会创建新的数组**:

In [11]:
a = mx.nd.ones((2, 2))
b = mx.nd.ones(a.shape)

b += a

b.asnumpy()

array([[ 2.,  2.],
       [ 2.,  2.]], dtype=float32)

#### 索引和切片

[]作用于第一个维度，axis=0,

In [12]:
a = mx.nd.array(np.arange(6).reshape(3,2))

a[1:2] = 1
a[:].asnumpy()

array([[ 0.,  1.],
       [ 1.,  1.],
       [ 4.,  5.]], dtype=float32)

如何对特定axis 切片呢？使用slice_axis方法:

In [13]:
d = mx.nd.slice_axis(a, axis=1, begin=1, end=2)
d.asnumpy()

array([[ 1.],
       [ 1.],
       [ 5.]], dtype=float32)

#### 改变数组的shape

只要不改变数组的size，就可以随意改变数组的shape:

In [14]:
a = mx.nd.array(np.arange(24))
b = a.reshape((2, 3, 4))
b.asnumpy()

array([[[  0.,   1.,   2.,   3.],
        [  4.,   5.,   6.,   7.],
        [  8.,   9.,  10.,  11.]],

       [[ 12.,  13.,  14.,  15.],
        [ 16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.]]], dtype=float32)

concatenate方法可以将多个shape相同的数组合并为一个数组，方式是沿着第１维度堆积(stack):

In [15]:
a = mx.nd.ones((2,3))
b = mx.nd.ones((2,3)) * 2
c = mx.nd.concatenate([a, b])
c.asnumpy()

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 2.,  2.,  2.],
       [ 2.,  2.,  2.]], dtype=float32)

#### Reduce

我们可以把一个数组reduce成一个常数:

In [16]:
a = mx.nd.ones((2,3))
b = mx.nd.sum(a) # 注意, b仍是NDArray对象
b.asnumpy()

array([ 6.], dtype=float32)

也可以只计算某个维度的和,

In [17]:
c = mx.nd.sum_axis(a, axis=1)
c.asnumpy()

array([ 3.,  3.], dtype=float32)

#### 广播

通过复制来广播一个数组，

In [18]:
a = mx.nd.array(np.arange(6).reshape(6,1))
b = a.broadcast_to((6, 2))
b.asnumpy()

array([[ 0.,  0.],
       [ 1.,  1.],
       [ 2.,  2.],
       [ 3.,  3.],
       [ 4.,  4.],
       [ 5.,  5.]], dtype=float32)

**广播也可以用于* 和 + 等操作**:

In [19]:
a = mx.nd.ones((3,2))
b = mx.nd.ones((1,2))

c = a + b


c.asnumpy()

array([[ 2.,  2.],
       [ 2.,  2.],
       [ 2.,  2.]], dtype=float32)

#### 复制

NDArray，正常的赋值操作并不会出现数据复制，仅仅是创建一个引用:

In [20]:
a = mx.nd.ones((2,3))
b = a

a is b

True

给函数传参数时也不会拷贝，仅仅是引用:

In [21]:
def f(x):
    return x

a is f(a)

True

copy方法会创建一个数组，并且deep copy，

In [22]:
b = a.copy()

b is a


False

如何不创建新的数组，还能copy某个数组的值呢？可以用[]或copyto()

In [23]:
# 先创建一个新的数组
b = mx.nd.ones(a.shape)
c = b
c[:] = a # 将a的值复制给c

d = b
a.copyto(d) # 将a的值复制给d

(c is b, d is b)

(True, True)

### 高级特性

mxnet.ndarray除了以上的基本操作，还包含一些高级操作，使得mxnet区别于一般的库

#### 支持GPU

默认使用CPU, 但很容易使用GPU哦。设备信息存储在ndarray.context。只要机器有Nvidia GPU显卡并且MXNet编译时USE_CUDA=1，我们就可以使用CPU 0来操作NDArray, mx.gpu(0)或简写为mx.gpu(). 如果有多块显卡，比如第2块显卡表示为mx.gpu(1).

In [24]:
def f():
    a = mx.nd.ones((100, 100))
    b = mx.nd.ones((100, 100))
    c = a + b
    print(c)

# 默认使用mx.cpu()
f()

# 使用gpu
with mx.Context(mx.gpu()):
    f()

<NDArray 100x100 @cpu(0)>
<NDArray 100x100 @gpu(0)>


在创建NDArray时，我们可以指定数组的context:

In [25]:
a = mx.nd.ones((100, 100), mx.gpu(0))
a

<NDArray 100x100 @gpu(0)>

**注意，如果两个数组位于不同的设备，他们是不能进行计算的**。所以如何将某个设备的数组拷贝到其他设备，来进行计算呢？

In [26]:
a = mx.nd.ones((100, 100), mx.cpu())
b = mx.nd.ones((100, 100), mx.gpu())
c = mx.nd.ones((100, 100), mx.gpu())
a.copyto(c) # 从CPU拷贝到GPU ????????????????????????????????????????????????????????????????
d = b + c

e = b.as_in_context(c.context) + c
{'d': d, 'e': e}

{'d': <NDArray 100x100 @gpu(0)>, 'e': <NDArray 100x100 @gpu(0)>}

#### 序列化

有两种方法实现序列化，一种是使用pickle:

In [27]:
import pickle as pkl

a = mx.nd.ones((2, 3))
# pack and then dump into disk
data = pkl.dumps(a)
pkl.dump(data, open('tmp.pickle', 'wb'))
#load from disk then unpack
data = pkl.load(open('tmp.pickle', 'rb'))
b = pkl.loads(data)

b.asnumpy()

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.]], dtype=float32)

第二种方法使用save和load方法, 比pickle更好:

In [28]:
a = mx.nd.ones((2,3))
b = mx.nd.ones((5,6))
mx.nd.save("temp.ndarray", [a, b]) #  序列化一个list

c = mx.nd.load("temp.ndarray")
c

[<NDArray 2x3 @cpu(0)>, <NDArray 5x6 @cpu(0)>]

还可以序列化一个dict:

In [29]:
d = {'a': a, 'b': b}
mx.nd.save("temp.ndarray", d)

c = mx.nd.load("temp.ndarray")
c

{'a': <NDArray 2x3 @cpu(0)>, 'b': <NDArray 5x6 @cpu(0)>}

为什么说save/load 比pickle好呢？优点有两个:

* 跨语言，比如Python接口序列化的数据，使用R接口可以读取
* 支持分布式系统文件，比如Amazon S3或Hadoop HDF5

#### lazy evaluation 和自动并行化

 MXNet使用lazy evaluation来获得更好的性能。当在python中运行a=b+1时，python线程仅仅是把这个操作push给后端引擎然后就直接返回（而不是立即执行a=b+1）。这种优化的好处是:
 
 * python线程可以继续执行其他的操作，一旦上一个操作push给后端引擎。减少了前端语言的负担。
 * 对于后端引擎来说，可以使用一些优化手段，比如自动并行化。

后端引擎能够解决数据依赖性(data dependencies)和正确调度计算过程(schedule)。对于前端用户来说，后端引擎是透明的。我们可以对存储计算结果的数组显示地调用wait_to_read 方法来等待计算完成。实际上，asnumpy等方法就隐式地调用了wait_to_read。